# Segment Anything Model as Inference

In [1]:
from skimage.segmentation import watershed, felzenszwalb
from skimage.filters import sobel
import pandas as pd
from pathlib import Path
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from skimage.filters import rank
from scipy import ndimage as ndi
from skimage.morphology import disk
import sklearn.metrics

In [3]:
# Load the train labels
# Note the transpose!
data_dir = Path("./../../data/")
labels_train = pd.read_csv(data_dir  / "Y_train.csv", index_col=0).T

In [4]:
# Here is a function to load the data
def load_dataset(dataset_dir):
    dataset_list = []
    # Note: It's very important to load the images in the correct numerical order!
    for image_file in list(sorted(Path(dataset_dir).glob("*.png"), key=lambda filename: int(filename.name.rstrip(".png")))):
        dataset_list.append(cv2.imread(str(image_file), cv2.IMREAD_GRAYSCALE))
    return np.stack(dataset_list, axis=0)

In [6]:
# Load the train and test sets
# If you've put the shortcut directly in your drive, this should work out of the box
# Else, edit the path
data_dir = Path("./../../data/")
data_train = load_dataset(data_dir / "X_train")
data_test = load_dataset(data_dir / "X_test")

In [7]:
# The train data is a numpy array of 1000 images of 512*512
print(f"X_train shape: {data_train.shape}")
# The train label is a dataframe of 1000 rows with 262144 (=512x512) columns
print(f"Y_train shape: {labels_train.shape}")

X_train shape: (1000, 512, 512)
Y_train shape: (1000, 262144)


In [8]:
from segment_anything import SamAutomaticMaskGenerator, sam_model_registry
sam = sam_model_registry["vit_l"](checkpoint="/Users/sarrabenyahia/Documents/GitHub/MedSAM/checkpoints_sam/sam_vit_l_0b3195.pth")

In [11]:
image = cv2.imread('./../../data/X_test/0.png')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

mask_generator = SamAutomaticMaskGenerator(sam)
masks = mask_generator.generate(image)

def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)
    polygons = []
    color = []
    for ann in sorted_anns:
        m = ann['segmentation']
        img = np.ones((m.shape[0], m.shape[1], 3))
        color_mask = np.random.random((1, 3)).tolist()[0]
        for i in range(3):
            img[:,:,i] = color_mask[i]
        ax.imshow(np.dstack((img, m*0.35)))
        
plt.figure(figsize=(7, 7))
plt.imshow(image)
show_anns(masks)
plt.axis('off')
plt.show() 

In [ ]:
masks

[{'segmentation': array([[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]]),
  'area': 179166,
  'bbox': [29, 0, 425, 511],
  'predicted_iou': 1.0851157903671265,
  'point_coords': [[312.0, 120.0]],
  'stability_score': 0.9615581035614014,
  'crop_box': [0, 0, 512, 512]},
 {'segmentation': array([[False, False, False, ...,  True,  True,  True],
         [False, False, False, ...,  True,  True,  True],
         [False, False, False, ...,  True,  True,  True],
         ...,
         [False, False, False, ...,  True,  True,  True],
         [False, False, False, ...,  True,  True,  True],
         [False, False, False, ...,  True,  True,  True]]),
  'area': 46348,
  'bbox': [330, 0, 181, 511

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd

# Replace this with the path to your X_test directory
x_test_dir = './../data/X_test/'

# Create the predictions directory if it does not exist
predictions_dir = './../../data/predictions/inference_sam'
os.makedirs(predictions_dir, exist_ok=True)

# Load the SamAutomaticMaskGenerator object
sam = sam_model_registry["vit_l"](checkpoint="/Users/sarrabenyahia/Documents/GitHub/MedSAM/checkpoints_sam/sam_vit_l_0b3195.pth")

# Loop over each image in the directory
for filename in os.listdir(x_test_dir):
    # Read the image and convert it to RGB
    image = cv2.imread(os.path.join(x_test_dir, filename))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Generate the mask using the SamAutomaticMaskGenerator object
    mask_generator = SamAutomaticMaskGenerator(sam)
    masks = mask_generator.generate(image)

    # Plot the image and mask
    plt.figure(figsize=(7, 7))
    plt.imshow(image)
    show_anns(masks)
    plt.axis('off')
    plt.show()


TypeError: unsupported operand type(s) for *: 'dict' and 'int'

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd

# Replace this with the path to your X_test directory
x_test_dir = './../../data/X_test/'

# Create the predictions directory if it does not exist
predictions_dir = './../../data/predictions/inference_sam'
os.makedirs(predictions_dir, exist_ok=True)

# Load the SamAutomaticMaskGenerator object
sam = sam_model_registry["vit_l"](checkpoint="/Users/sarrabenyahia/Documents/GitHub/MedSAM/checkpoints_sam/sam_vit_l_0b3195.pth")

# Loop over each image in the directory
for filename in os.listdir(x_test_dir):
    # Read the image and convert it to RGB
    image = cv2.imread(os.path.join(x_test_dir, filename))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Generate the mask using the SamAutomaticMaskGenerator object
    mask_generator = SamAutomaticMaskGenerator(sam)
    masks = mask_generator.generate(image)

    # Save the mask to a file
    mask_path = os.path.join(predictions_dir, filename)
    mask = np.uint8(masks[0] * 255)
    cv2.imwrite(mask_path, mask)
